In [25]:
import pandas as pd
from collections import deque

In [26]:
plays = pd.read_csv('../data/play_by_play.txt', delimiter = "\t")
lineup = pd.read_csv('../data/lineup.txt', delimiter = "\t").sort_values(['Game_id', 'Period', 'Team_id'], 
                                                                         ascending = [True, True, True])
plays = plays.sort_values(['Game_id', 'Period', 'PC_Time', 'WC_Time', 'Event_Num'], 
                 axis = 0, ascending = [True, True, False, True, True]).drop(['Team_id_type', 'Option2', 'Option3'], axis = 1)
game_ids = plays['Game_id'].unique().tolist()

#working with game 0
lineup0 = lineup[lineup.Game_id == game_ids[0]].reset_index()
df = plays[plays.Game_id == game_ids[0]]

# something with views if error in future

In [27]:
team1players = {}
team2players = {}



t1Active = []
t2Active = []

t1Active2=[]
t1Active3=[]
t1Active4=[]

t2Active2=[]
t2Active3=[]
t2Active4=[]

team1 = ''
team2 = ''

# bigList1 = []
# bigList2 = []


for i, row in lineup0.iterrows(): #0-9, 10-19, 20-29, 30-30
    if i < 10:
        if i < 5:
            team1players[row['Person_id']] = 0
            t1Active.append(row['Person_id'])
            team1 = row["Team_id"]
            
        else:
            team2players[row['Person_id']] = 0
            t2Active.append(row['Person_id'])
            team2 = row["Team_id"]
            
    elif 10 <= i <= 19:
        if (i < 15):          
            team1players[row['Person_id']] = 0
            t1Active2.append(row['Person_id'])           
        else:
            team2players[row['Person_id']] = 0
            t2Active2.append(row['Person_id'])
    elif 20<= i<=29:
        if (i<25):
            team1players[row['Person_id']] = 0
            t1Active3.append(row['Person_id'])  
        else:
            team2players[row['Person_id']] = 0
            t2Active3.append(row['Person_id'])
    else:
        if (i<35):
            team1players[row['Person_id']] = 0
            t1Active4.append(row['Person_id'])  
        else:
            team2players[row['Person_id']] = 0
            t2Active4.append(row['Person_id'])
            
            
        
# bigList1.append(t1Active)
# bigList2.append(t2Active)
subsInFoul = deque()
flag = False

#print(df.loc[147:157])
for i, event in df.iterrows():

    
    event_type = event['Event_Msg_Type']
    
    #6,8,1 done, 2,4,5,10,7,9,11 don't matter
    #
    #1-made shot, 2-missed shot, 3-free throw, 4-rebound, 5-turnover, 6-foul, 7-violation, 8-substitution, 
    #9-Timeout, 10-Jump Ball, 11-Ejection, 12-Start Period, 13-End Period
    
    if event_type==12:
        period = event['Period']
        if (period == 2):
            t1Active = t1Active2
            t2Active = t2Active2
        elif (period == 3):
            t1Active = t1Active3
            t2Active = t2Active3
        elif (period==4):
            t1Active=t1Active4
            t2Active=t2Active4
    
    if event_type == 6:
        flag = True
        #is a free throw happening
        
    if event_type == 8: 
        person1 = event['Person1']
        person2 = event['Person2']
#        print(person1, person2, event['Team_id'])    
        if not flag: #if free throw is definitely not happening
        
            if person1 in team1players:
                t1Active.remove(person1)
                t1Active.append(person2)
                if person2 not in team1players:
                    team1players[person2] = 0 #add them to the dictionary
            
            else:
                t2Active.remove(person1)
                t2Active.append(person2)
                if person2 not in team2players:
                    team2players[person2] = 0
                    
                    
        else: #free throw might be happening, add them to the queue
#            subsInFoul.append((person1, person2,event['Team_id']))

            subsInFoul.append( (person1, person2))
    
    if (flag == True) and (event_type == 1 or event_type == 2 or event_type == 4 or event_type == 5 or event_type == 10 
                           or event_type==7):
        #play went on, foul did not result in free throws
        flag = False

        
        while len(subsInFoul) != 0:
            
          
            sub = subsInFoul.popleft()
#            print(sub)
            if sub[0] in team1players:
                t1Active.remove(sub[0])
                t1Active.append(sub[1])
                if sub[1] not in team1players:
                    team1players[sub[1]] = 0
                    
            else:
                t2Active.remove(sub[0])
                t2Active.append(sub[1])
                if sub[1] not in team2players:
                    team2players[sub[1]] = 0
                    
        
                    
    #compute plus minus
    
    if (flag == False) and (event_type==1):
        person1 = event['Person1']
        person2 = event['Person2']
        
        if (person1 in team1players):
            for i in t1Active:
                team1players[i] += event['Option1']

            for i in t2Active:
                team2players[i] -= event['Option1']                          
        else:
            for i in t1Active:
                team1players[i] -= event['Option1']
            for i in t2Active:
                team2players[i] += event['Option1']          
                        
    if (event_type==3):
        
        person1 = event['Person1']
        person2 = event['Person2']
        
        if (person1 in team1players):
            for i in t1Active:
                team1players[i] += event['Option1']
            for i in t2Active:
                team2players[i] -= event['Option1']                        
        else:
            for i in t1Active:
                team1players[i] -= event['Option1']
            for i in t2Active:
                team2players[i] += event['Option1']
            
           

print(team1players)
print(team2players)

{'881f83d2dee3f18c7d1751659406144e': 2, 'cec898a1d355dbfbad8c760615fde1af': 4, 'a99f44bbff39e352191a870e17f04537': 24, '89706b99ddd00dc05d37ef5cafc04276': 15, '2b313e2bcef0268bc8e9415132ba9997': -5, '2d9594e4d250679e7a3383ad5ccd2f44': -2, 'd6bc5eb6ed54c734628cb4d237f99031': 1, 'b2fada91a0c8a0f400623c425286ae2e': 7, '1362d9ad805e7cc4ea3a9467c91142d6': 1, '314c71574689f752c17b83cd6a4d770e': -7}
{'27ea17a8685c4919f157e83fe9cb2d9e': -10, '57bbd7e30bc694aeee9ee40c583e6811': -15, '33963fe856a1523ff46438ba07d1d99f': 1, 'c00264c3114d23bac482e9de50fb7d28': -10, '307beab25b1021a548b4a47550bc4b25': -9, '8bb18fb1d0cc10c985afa823ecec6d18': -1, 'c1fe6ed491d04e3cdf50755dff23b263': 9, '603daefb2fa321f3abab289805fdcc25': -7, '60b5f9f68ccbbf7ba3fa13a2189e2775': 2}
